In [1]:
import pandas as pd

## Clean LTC data

In [2]:
# convert name and additional info columns to lowercase
df_ngan = pd.read_csv('df_final_ngan.csv')
df_ngan['cleaned_name'] = df_ngan['name'].str.lower()
df_ngan['additional_info'] = df_ngan['additional_info'].str.lower()

# adjust home names to match LTC COVID data
df_ngan["cleaned_name"].replace({"vision '74 inc.": "vision nursing home", "finlandia hoivakoti nursing home limited": "finlandia hoivakoti nursing home", "pinecrest nursing home - bobcaygeon": "pinecrest nursing home (bobcaygeon)", "residence saint- louis": "residence saint-louis", "st patrick's home": "st. patrick's home"}, inplace=True)
df_ngan.head()

,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,5y_inspections,cleaned_name
0,AFTON PARK PLACE LONG TERM CARE COMMUNITY,1200 Afton Drive,"Sarnia, N7S6L6",Erie St. Clair,S & R Nursing Homes Ltd.,NaN,For-Profit,Home with approximately 128 beds,No,Yes,Yes,Yes,NaN,Sarnia,N7S6L6,128.0,67.0,34.0,afton park place long term care community
1,"ALBRIGHT GARDENS HOMES, INCORPORATED",5050 Hillside Drive,"Beamsville, L0R1B2",Hamilton Niagara Haldimand Brant (Hnhb),"Albright Gardens Homes, Incorporated",NaN,Non-Profit,Home with approximately 231 beds,No,Yes,Yes,No,NaN,Beamsville,L0R1B2,231.0,39.0,25.0,"albright gardens homes, incorporated"
2,ALEXANDER PLACE,329 Parkside Drive P. O. Box 50,"Waterdown, L0R2H0",Hamilton Niagara Haldimand Brant (Hnhb),Waterdown Long Term Care Centre Inc.,NaN,For-Profit,Home with approximately 128 beds,Yes,Yes,Yes,Yes,NaN,Waterdown,L0R2H0,128.0,28.0,17.0,alexander place
3,ALGOMA DISTRICT HOMES FOR THE AGED (ALGOMA MANOR),135 Dawson Street,"Thessalon, P0R1L0",North East,Board Of Management For The District Of Algoma,NaN,NaN,Home with approximately 108 beds,Yes,Yes,Yes,No,home closed on 11/01/2011,Thessalon,P0R1L0,108.0,5.0,0.0,algoma district homes for the aged (algoma manor)
4,ALGOMA MANOR NURSING HOME,145 Dawson Street,"Thessalon, P0R1L0",North East,Algoma Manor Nursing Home,NaN,NaN,Home with approximately 96 beds,Yes,Yes,Yes,Yes,NaN,Thessalon,P0R1L0,96.0,23.0,14.0,algoma manor nursing home


## Review records of homes that have been closed

In [3]:
# we see that a 20 homes are closed, and one home was merged
df_ngan['additional_info'].value_counts()
df_ngan.additional_info.str.contains("closed", na=False).value_counts()

False    631
True      20
Name: additional_info, dtype: int64

In [4]:
# drop rows with homes that are closed
df_ngan = df_ngan[~df_ngan.additional_info.str.contains("closed", na=False)].reset_index()
df_ngan['additional_info'].value_counts()

datars bere, dearness home managing director                                                                                                                                     1
recipient no 346524 – sarsfield colonial home\r\ncomment - change in ownership effective february 19 2020. details transferred to recipient no 664183/ facility no nh1692\r\n    1
stutti@craigwielgardens.on.ca                                                                                                                                                    1
home was merged with mount hope long-term care centre effective 1st january, 2016. please see mount hope long-term care centre for information.                                  1
pavillon omer deslauriers, 37 beds, long term care services provided in french and english.                                                                                      1
Name: additional_info, dtype: int64

## Manual Review of one home that was merged in Jan 2016

- Address to the merged home is just a general PO BOX to the licensee
- Will remove the merged home from the database

In [5]:
df_ngan[df_ngan['additional_info'].str.contains('hope', regex=False, na=False)]

,index,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,5y_inspections,cleaned_name
487,504,"ST. JOSEPH'S HEALTH CARE, LONDON - MOUNT HOPE ...",200 College Avenue P.O. Box 5777,"London, N6A1Y1",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 217 beds,Yes,Yes,Yes,Yes,home was merged with mount hope long-term care...,London,N6A1Y1,217.0,68.0,17.0,"st. joseph's health care, london - mount hope ..."


In [6]:
# show rows with information on the 2 homes that were merged
df_ngan[df_ngan['cleaned_name'].str.contains('mount hope', regex=False, na=False)]

,index,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,5y_inspections,cleaned_name
360,373,MOUNT HOPE CENTRE FOR LONG TERM CARE,21 Grosvenor Street P.O. Box 5777,"London, N6A1Y6",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 177 beds,Yes,Yes,Yes,Yes,NaN,London,N6A1Y6,177.0,87.0,44.0,mount hope centre for long term care
487,504,"ST. JOSEPH'S HEALTH CARE, LONDON - MOUNT HOPE ...",200 College Avenue P.O. Box 5777,"London, N6A1Y1",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 217 beds,Yes,Yes,Yes,Yes,home was merged with mount hope long-term care...,London,N6A1Y1,217.0,68.0,17.0,"st. joseph's health care, london - mount hope ..."


In [7]:
df_ngan.iloc[487]

index                                                                 504
name                    ST. JOSEPH'S HEALTH CARE, LONDON - MOUNT HOPE ...
address                                  200 College Avenue P.O. Box 5777
city and postal code                                       London, N6A1Y1
LHIN                                                           South West
licensee                                 St. Joseph's Health Care, London
management                                                            NaN
home type                                                      Non-Profit
beds                                     Home with approximately 217 beds
short stay                                                            Yes
residents council                                                     Yes
family council                                                        Yes
accreditation                                                         Yes
additional_info         home was merge

In [8]:
# drop row with the merged home
df_ngan = df_ngan.drop(df_ngan.index[487])
# check to see it is gone
df_ngan[df_ngan['cleaned_name'].str.contains('mount hope', regex=False, na=False)]

,index,name,address,city and postal code,LHIN,licensee,management,home type,beds,short stay,residents council,family council,accreditation,additional_info,city,postal code,number of beds,total_inspections,5y_inspections,cleaned_name
360,373,MOUNT HOPE CENTRE FOR LONG TERM CARE,21 Grosvenor Street P.O. Box 5777,"London, N6A1Y6",South West,"St. Joseph's Health Care, London",NaN,Non-Profit,Home with approximately 177 beds,Yes,Yes,Yes,Yes,NaN,London,N6A1Y6,177.0,87.0,44.0,mount hope centre for long term care


## Merge LTC and LTC covid data

In [9]:
df_kt = pd.read_csv('merged_ltc.csv')
df_kt['cleaned_name'] = df_kt['LTC Home'].str.lower()
df_kt.head()

,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,cleaned_name
0,0,Almonte Country Haven,Almonte,82,<5,28,13,Active,almonte country haven
1,1,Altamont Care Community,Scarborough,159,72,46,60,Active,altamont care community
2,2,Anson Place Care Centre,Hagersville,61,28,23,29,Active,anson place care centre
3,3,Arbour Creek Long-Term Care Centre,Hamilton,129,0,0,<5,Active,arbour creek long-term care centre
4,4,Avalon Retirement Centre,Orangeville,137,0,0,<5,Active,avalon retirement centre


In [10]:
a = set(df_ngan['cleaned_name'])
b = set(df_kt['cleaned_name'])
def returnNotMatches(a, b):
    return [[x for x in b if x not in a]]
returnNotMatches(a, b)

[[]]

In [11]:
df_merged = pd.merge(left=df_ngan, right=df_kt, how='outer', left_on='cleaned_name', right_on='cleaned_name')

In [12]:
df_merged.to_csv(r'df_ltc_final.csv', index = False)

## Import and prep ODHF data

In [31]:
odhf = pd.read_csv('odhf_v1.csv', engine='python')
df_on = odhf[odhf['province'].isin(['on'])].reset_index()

import unicodedata
df_on['cleaned_name'] = df_on['facility_name'].str.lower().apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())
df_on.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3733 entries, 0 to 3732
Data columns (total 19 columns):
level_0                      3733 non-null int64
index                        3733 non-null int64
facility_name                3733 non-null object
source_facility_type         3705 non-null object
odhf_facility_type           3728 non-null object
provider                     3733 non-null object
unit                         0 non-null object
street_no                    3122 non-null object
street_name                  3154 non-null object
postal_code                  3726 non-null object
city                         3724 non-null object
province                     3733 non-null object
source_format_str_address    271 non-null object
CSDname                      3540 non-null object
CSDuid                       3151 non-null float64
Pruid                        3733 non-null int64
latitude                     3159 non-null float64
longitude                    3159 non-null float

In [38]:
df_on[df_on['cleaned_name'].str.contains('stratford', regex=False, na=False)]

,level_0,index,facility_name,source_facility_type,odhf_facility_type,provider,unit,street_no,street_name,postal_code,city,province,source_format_str_address,CSDname,CSDuid,Pruid,latitude,longitude,cleaned_name
514,2914,2915,cedarcroft stratford retirement residence,retirement home,Nursing and residential care facilities,Province of Ontario,NaN,260,church street,N5A2R6,stratford,on,NaN,Stratford,3531011.0,35,43.364941,-80.990390,cedarcroft stratford retirement residence
832,3232,3233,community health services - canadian red cross...,community support service,Ambulatory health care services,Province of Ontario,NaN,100,gordon street,N5A7T8,stratford,on,NaN,Stratford,3531011.0,35,43.371969,-80.960394,community health services - canadian red cross...
1546,3946,3947,huron perth healthcare alliance (head office) ...,general,Hospitals,Public Health Agency of Canada,NaN,46,general hospital dr,N5A2Y6,stratford,on,46 general hospital dr. stratford on n5a 2y6,Stratford,3531011.0,35,43.368168,-80.995481,huron perth healthcare alliance (head office) ...
1550,3950,3951,huron perth healthcare alliance stratford site...,community support service,Ambulatory health care services,Province of Ontario,NaN,90,john street s,N5A2Y8,stratford,on,NaN,Stratford,3531011.0,35,43.368188,-80.997159,huron perth healthcare alliance stratford site...
1551,3951,3952,huron perth healthcare alliance � stratford ge...,"active acute hospital, long term care",Hospitals,Canadian Institute for Health Information,NaN,NaN,NaN,N5A2Y6,stratford,on,NaN,Stratford,NaN,35,NaN,NaN,huron perth healthcare alliance stratford gen...
2349,4749,4750,peoplecare � stratford,long term care,Nursing and residential care facilities,Canadian Institute for Health Information,NaN,NaN,NaN,N5A5G3,stratford,on,NaN,Stratford,NaN,35,NaN,NaN,peoplecare stratford
3089,5489,5490,stratford family health team,family health team - contract,Ambulatory health care services,Province of Ontario,NaN,342,erie street,N5A2N4,stratford,on,NaN,Stratford,3531011.0,35,43.363598,-80.989960,stratford family health team
3090,5490,5491,stratford general hospital,midwifery clinic,Ambulatory health care services,Province of Ontario,NaN,46,general hospital drive,N5A2Y6,stratford,on,NaN,Stratford,3531011.0,35,43.368176,-80.995485,stratford general hospital


In [40]:
home = df_on.iloc[[2349]]
home['cleaned_name']

2349    peoplecare  stratford
Name: cleaned_name, dtype: object

## List of homes in the LTC database that are not listed in the ODHF

In [15]:
a = set(df_on['cleaned_name'])
b = set(df_merged['cleaned_name'])
def returnNotMatches(a, b):
    return [x for x in b if x not in a]
missing_odhf = returnNotMatches(a, b)

In [16]:
len(missing_odhf)


24

In [17]:
missing_odhf.sort()
missing_odhf

['albright gardens homes, incorporated',
 'bella senior care residences inc.',
 'bon air long term care residence',
 'caressant care cobden',
 'caressant care harriston',
 'champlain long term care residence',
 'heartwood (fka versa-care cornwall)',
 'lady isabelle nursing home',
 'lancaster long term care residence',
 'malden park continuing care centre',
 'niagara long term care residence',
 'north renfrew long-term care services',
 'north shore health network – eldcap unit',
 'north shore health network – ltc unit',
 'people care centre',
 'residence saint-louis',
 'santé manitouwadge health',
 "st. joseph's health centre, guelph",
 "st. joseph's mother house (martha wing)",
 "st. joseph's villa, dundas",
 'strathcona long term care',
 'the meadows',
 'william a. "bill" george extended care facility',
 'élisabeth-bruyère residence']

## Replace cleaned names in the LTC and ODHF df

In [41]:
df_merged_names = df_merged['cleaned_name'].replace({
    'albright gardens homes, incorporated' : 'albright gardens',
    'bella senior care residences inc.':'bella senior care residences',
    'bon air long term care residence':'chartwell bon air long term care residence',
    'caressant care cobden':'caressant care - cobden',
    'caressant care harriston':'caressant care - harriston',
    'champlain long term care residence':'chartwell champlain long term care residence',
    'heartwood (fka versa-care cornwall)':'heartwood',
# lady isabelle MISSING, SEE NOTE 
    'lancaster long term care residence':'chartwell lancaster long term care residence',
# malden MISSING, SEE NOTE 
    'niagara long term care residence':'chartwell niagara long term care residence',
    'north renfrew long-term care services':'north renfrew long-term care services inc.',
    'north shore health network – eldcap unit':'north shore health network - eldcap unit' ,
    'north shore health network – ltc unit' : 'north shore health network - ltc unit',
    'people care centre':'peoplecare  stratford',
# residence saint-louis, OK
    'santé manitouwadge health':'sante manitouwadge health',
    'st. joseph\'s health centre, guelph':'st. joseph\'s health centre - guelph',
    'st. joseph\'s mother house (martha wing)':'st. josephs motherhouse martha wing',
    'st. joseph\'s villa, dundas' : "st. joseph's villa (dundas)",
# strathcona long term care, MISSING SEE NOTE
    'the meadows':'revera inc.  the meadows long term care centre',
    'william a. "bill" george extended care facility': 'william a. \'bill\' george extended care facility',
    'élisabeth-bruyère residence':'elisabeth-bruyere residence'
})

df_on_names = df_on['cleaned_name'].replace({'lisabeth-bruyre residence': 'elisabeth-bruyere residence',
                                             'rsidence saint-louis' : 'residence saint-louis',
                                             'peoplecare stratford':'people care centre'})






In [42]:
a = set(df_on_names)
b = set(df_merged_names)
def returnNotMatches(a, b):
    return [x for x in b if x not in a]
missing_odhf_names = returnNotMatches(a, b)
print(len(missing_odhf_names))
missing_odhf_names

3


['strathcona long term care',
 'lady isabelle nursing home',
 'malden park continuing care centre']

## Manual review
- lady isabelle nursing home: http://www.ladyisabelle.ca/Contact_Information.html, MISSING from ODHF
- MALDEN PARK CONTINUING CARE CENTRE: seems to be attached to **Windsor Regional Hospital** which is in ODHF if we need the geolocation data. **hotel-dieu grace healthcare css** has same location in ODHF.
- people care centre: listed in ODHF but no geodata, https://www.peoplecare.ca/
- sante manitouwadge health: seems to be attached to manitouwadge general hospital
- Strathcona Long Term Care, https://svch.ca/long-term-care/strathcona/location/, in ODHF have saugeen valley nursing center which is at the same address, https://svch.ca/saugeen-valley-nursing-centre-announces-plan-to-redevelop-87-bed-facility-in-mount-forest/#.Xr2rvRNKjfY